# Introducción

Con este proyecto mi objetivo es desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline.

Observaré los datos de los clientes que heredaron sus planes y entrenaré un modelo a base de el comportamiento del cliente para hacerle una recomendación.

## 1 Voy a cargar las librerías y la tabla con la información de los clientes, además de ver la información de las mismas.

In [1]:
# Cargar librerías.

from scipy import stats as st
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Carga los datos en un DataFrame.

users = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# Mostrar parte del Dataframe

users.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
users.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
# Ver si hay duplicados

print(users[users.duplicated()])

Empty DataFrame
Columns: [calls, minutes, messages, mb_used, is_ultra]
Index: []


In [6]:
# Ver si hay valores nulos

users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## 2 Comenzaré a usar el dataset para entrenar los diferentes modelos y medir su exactitud.

In [11]:
# Segmentar el Dataframe en el de entrenamiento y validación

features= users.drop(['is_ultra'], axis=1)
target= users['is_ultra']



features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=54321)
features_train, features_val, target_train, target_val = train_test_split(features_train, target_train, test_size=0.2, random_state=54321)





In [12]:
# Entrenamiento del modelo de árbol de decisión y ver su exactitud.

best_depth = 0
best_score = 0
for depth in range(1, 11):  # selecciona el rango del hiperparámetro
    model = DecisionTreeClassifier(random_state=54321, max_depth = depth)
    model.fit(features, target)
    train_predictions = model.predict(features)
    test_predictions = model.predict(features_test)
   
    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", accuracy_score(target, train_predictions)) # calcula la puntuación de accuracy en el conjunto de entrenamiento
    print("Conjunto de prueba:", accuracy_score(target_test, test_predictions)) # calcula la puntuación de accuracy en el conjunto de prueba
    
print()

model = DecisionTreeClassifier(random_state=54321, max_depth = 10)
model.fit(features, target)
val_predictions = model.predict(features_val)
print("Conjunto de validación:", accuracy_score(target_val, val_predictions))

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.752644679527069
Conjunto de prueba: 0.7247278382581649
Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.7840696950840075
Conjunto de prueba: 0.7620528771384136
Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.798070939639079
Conjunto de prueba: 0.7698289269051322
Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.7999377722464219
Conjunto de prueba: 0.7713841368584758
Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.8170504044803982
Conjunto de prueba: 0.7978227060653188
Exactitud de max_depth igual a 6
Conjunto de entrenamiento: 0.827317983820784
Conjunto de prueba: 0.8040435458786936
Exactitud de max_depth igual a 7
Conjunto de entrenamiento: 0.8397635345364033
Conjunto de prueba: 0.807153965785381
Exactitud de max_depth igual a 8
Conjunto de entrenamiento: 0.8550093341630367
Conjunto de prueba: 0.8242612752721618
Exactitud de max_depth igual a 9
Conjunto de entrenamiento: 

In [13]:
# Entrenamiento del modelo de regresión logística con su prueba de exactitud.

model = LogisticRegression(random_state=54321, solver='liblinear') # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features, target) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features, target) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_test, target_test) # calcula la puntuación de accuracy en el conjunto de validación
score_test = model.score(features_val, target_val)
print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de prueba:", score_valid)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_test)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7426882389545737
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7060653188180405
Accuracy del modelo de regresión logística en el conjunto de prueba: 0.7514563106796116


In [19]:
# Entrenamiento del modelo de árboles aleatorios con su prueba de exactitud.
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(features, target) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_test, target_test) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

        
model = RandomForestClassifier(random_state=54321, n_estimators=9) # configura el número de árboles
model.fit(features, target) # entrena el modelo en el conjunto de entrenamiento
score = model.score(features_val, target_val) # calcula la puntuación de accuracy en el conjunto de validación
print("La exactitud del mejor modelo en el conjunto de prueba (n_estimators = {}): {}".format(best_est, best_score))
print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = 9):", score)

La exactitud del mejor modelo en el conjunto de prueba (n_estimators = 9): 0.9782270606531882
La exactitud del mejor modelo en el conjunto de validación (n_estimators = 9): 0.9902912621359223


## 3 Conclusión general

En este punto observando las diferentes puntaciones en la exactitud, determino que el modelo con mejor puntuación de exactitud para clasificar a los clientes en el plan más conveniente es el de árboles aleatorios, en este caso al ser un modelo que busca clasificar entre dos planes ocupé los modelos que se usan para clasificar y no para la regresión.